In [3]:
# General 
import sys
import math
import time
import numpy as np
# import pandas as pd
#Plotting
# import matplotlib.pyplot as plt
# from mpl_toolkits import mplot3d
# from matplotlib import cm
#GPyTorch
import torch
import gpytorch
from torch.utils.data import TensorDataset, DataLoader
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
# import tqdm
# #Sklearn 
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import mean_squared_error

In [2]:
class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ZeroMean()
        #self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2))
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=2))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [3]:
class Data:
    def __init__(self, X=None, y=None):
        self.X = X
        self.y = y
    def set_data(self, X, y):
        self.X = X
        self.y = y
        self.read_data_torch()
    def read_data_torch(self):
        self.X_train =  torch.FloatTensor(self.X)
        self.y_train =  torch.FloatTensor(self.y)
        self.X_test =  torch.FloatTensor(self.X)
        self.y_test =  torch.FloatTensor(self.y)
    def read_data(self, inp):
        return

In [4]:
def create_grid(start, stop, res, array_type = None):
    x = np.linspace(start, stop, res)
    y = np.linspace(start, stop, res)
    x, y = np.meshgrid(x, y)
    if array_type is None:
        X = np.stack((x.flatten(), y.flatten()), axis=-1)
    else:
        X = torch.FloatTensor(np.stack((x.flatten(), y.flatten()), axis=-1))
    return X, x, y
X_grid, x_grid, y_grid = create_grid(-9.9,9.9,51)
X_grid_torch, x_grid_torch, y_grid_torch = create_grid(-9.9,9.9,51, "torch")

In [5]:
observed_pred = (model(X_grid_torch))
mean = observed_pred.mean.detach().numpy()
lower, upper = observed_pred.confidence_region()
lower = lower.detach().numpy()
upper = upper.detach().numpy()

NameError: name 'model' is not defined

In [79]:
y2 = np.zeros(mean.shape)
y = np.vstack((mean, y2))
y.shape

(2, 2601)

In [9]:
def plot_wind_field(X=None, y=None, res=8, title=None, name=None):
    fig, ax = plt.subplots(1, 1)
    plt.axis('equal')
    plt.grid(linestyle=':')
    if X is None:
        X = X
        y = y
        title = name
    q = plt.quiver(X[::res, 0], X[::res, 1], y[::res,0], y[::res,1], scale=2.54, color='r',
                   units='width', scale_units="inches", width=0.003)
    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.ioff()
    if not name==None:
        plt.savefig(name)
    plt.show()

In [22]:
plot_wind_field(X=X_grid, y=np.transpose(y), res=4)

NameError: name 'y' is not defined

# Compute the gradient

In [23]:
tic = time.time()
X = torch.autograd.Variable(torch.Tensor(X_grid_torch), requires_grad = True)
observed_pred = (model(X))
dydtest_x_ag = torch.autograd.grad(observed_pred.mean.sum(), X)[0]
y_grad = dydtest_x_ag.detach().numpy()
print(time.time() - tic)
print(y_grad)

NameError: name 'model' is not defined

In [24]:
plot_wind_field(X=X_grid, y=y_grad, res=4, title="Gradient of GP for wind in x")

NameError: name 'y_grad' is not defined

# Put together the prediction model

# 0.0) Nominal model

In [53]:
# Continuous time 
g = 9.81
tau = 0.075
k = 0.963
A = np.zeros([9,9])
A[0,3] = 1
A[1,4] = 1
A[2,5] = 1
A[3,7] = g
A[4,6] = -g
A[6,6] = -1/tau
A[7,7] = -1/tau
B = np.zeros([9,4])
B[5,3] = 1
B[6,0] = k/tau
B[7,1] = k/tau
B[8,2] = 1
Bd = np.zeros([9,2])
Bd[3,0] = 1
Bd[4,1] = 1

# Discrete time


In [54]:
from scipy.signal import cont2discrete, lti, dlti, dstep

C = np.identity(9)
D = np.zeros([9,4])
dt = 0.05
d_system = cont2discrete((A, B, C, D), dt, method='zoh')
A_d = d_system[0]
B_d = d_system[1]
C_d = d_system[2]
D_d = d_system[3]
with np.printoptions(precision=4, suppress=True):
    print(B_d)

[[ 0.      0.0022  0.      0.    ]
 [-0.0022  0.      0.      0.    ]
 [ 0.      0.      0.      0.0012]
 [ 0.      0.1276  0.      0.    ]
 [-0.1276  0.      0.      0.    ]
 [ 0.      0.      0.      0.05  ]
 [ 0.4686  0.      0.      0.    ]
 [ 0.      0.4686  0.      0.    ]
 [ 0.      0.      0.05    0.    ]]


In [55]:
#Also discretize the disturbance 
A_ = A
B_ = np.hstack([B, Bd])
C = np.identity(9)
D = np.zeros([9,4])
dt = 0.05
d_system = cont2discrete((A_, Bd, C, D), dt, method='zoh')
B_d_disc = d_system[1]
print(B_d_disc)

[[0.00125 0.     ]
 [0.      0.00125]
 [0.      0.     ]
 [0.05    0.     ]
 [0.      0.05   ]
 [0.      0.     ]
 [0.      0.     ]
 [0.      0.     ]
 [0.      0.     ]]


# 0.1) Load the trained GP models

In [7]:
data = np.load('/home/johanna/MasterThesis/data/backup_structured_x_wind/wind_data_4.npz')
data_X = data['X']
data_y = data['y']
inducing_points = torch.FloatTensor(data_X[::int(data_y.shape[0]/30), :])

In [8]:
inducing_points.size(0)

31

In [26]:
model = GPModel(inducing_points=inducing_points)
state_dict = torch.load('gpytorch_y1_model.pth')
model.load_state_dict(state_dict)
print(state_dict)

OrderedDict([('variational_strategy.inducing_points', tensor([[  0.6238,   1.6735],
        [  2.4993,  -1.8205],
        [  8.2422,  -0.1356],
        [  7.5675,   6.6363],
        [  2.2974,   9.2219],
        [ -6.2836,   9.3662],
        [ -5.9367,   1.5368],
        [ -4.8024,  -1.7412],
        [ -4.5350,  -3.1451],
        [ -8.9693,  -8.3928],
        [ -5.7966, -10.0445],
        [ -0.3767,  -9.9683],
        [  4.4215,  -9.5435],
        [  8.5375,  -9.7206],
        [  3.6561,  -5.3003],
        [ -1.5771,  -0.5819],
        [ -9.0627,  -1.9239],
        [ -9.8440,   0.3330],
        [ -9.4903,   3.4844],
        [ -8.9786,   6.7935],
        [ -2.4628,   5.3270],
        [  8.4513,   9.5683],
        [  9.6003,   3.7080],
        [  9.3990,  -3.3051],
        [  9.6274,  -5.7380],
        [  5.7499,   2.5925],
        [  1.9121,   4.7352],
        [ -8.7800,  -3.0534],
        [ -5.1636,  -4.1952],
        [ -2.0399,  -6.6442],
        [  6.2088,  -2.2447]])), ('variational

In [59]:
length_scale = (state_dict['covar_module.base_kernel.raw_lengthscale'].detach().numpy()[0])

In [60]:
model1 = GPModel(inducing_points=inducing_points)
state_dict = torch.load('gpytorch_y1_model.pth')
model1.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
model2 = GPModel(inducing_points=inducing_points)
state_dict = torch.load('gpytorch_y2_model.pth')
model2.load_state_dict(state_dict)

<All keys matched successfully>

In [14]:
# TODO: turn this into one function and move to ROS 

# 1) Retrieve mean prediction, standard deviation and gradient

In [15]:
def GP_predict(X):
    print(X)
    X_grad = torch.autograd.Variable(torch.Tensor(X), requires_grad = True)
    print(X_grad)
    observed_pred1 = (model1(X_grad))
    mean1 = observed_pred1.mean.detach().numpy()
    covar1 = observed_pred1.covariance_matrix.detach().numpy()
    dydtest_x_ag = torch.autograd.grad(observed_pred1.mean.sum(), X_grad)[0]
    y_grad1 = dydtest_x_ag.detach().numpy()
    observed_pred2 = (model2(X_grad))
    mean2 = observed_pred2.mean.detach().numpy()
    covar2 = observed_pred2.covariance_matrix.detach().numpy()
    dydtest_x_ag = torch.autograd.grad(observed_pred2.mean.sum(), X_grad)[0]
    y_grad2 = dydtest_x_ag.detach().numpy()
    y_mean = np.vstack((mean1, mean2))
    y_covar = np.diag([covar1[0,0], covar2[0,0]])
    y_grad = np.zeros([2,9])
    y_grad[:,0] = y_grad1
    y_grad[:,1] = y_grad2
    return y_mean, y_covar, y_grad
    #plot_wind_field(X=X_grid[1300:1301], y=np.transpose(y), res=1)

In [16]:
# Multiple points versus one point 
y_pred, y_covar, y_grad = GP_predict(X_grid[1300:1301])
y_pred.shape

[[0. 0.]]
tensor([[0., 0.]], requires_grad=True)
test
test
test
test
test
test


/home/johanna/.local/lib/python3.8/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:136: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2115.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution


(2, 1)

# 2) Initialize state and uncertainty

In [17]:
# Initialize state and uncertainty (uncertainty at the first state is zero)
x0 = np.zeros([9,1])
sigma_x0 = np.zeros([9,9])
u = np.zeros([4,1])

In [18]:
x0.shape

(9, 1)

# 3) Model prediction

In [19]:
def continuous_model(x_mean, x_covar, u):
    # Retrieve disturbance information 
    x_gp = np.transpose(x_mean[0:2,:])
    print(x_gp)
    d_mean, d_covar, d_grad = GP_predict(x_gp)
    print(d_covar)
    # Compute the mean of the next state
    x_full = np.vstack([x_mean, d_mean])
    print(x_full)
    x_i_next = np.matmul(np.hstack([A,Bd]), x_full) + np.matmul(B,u)
    # Compute the covariance of the next state 
    sigma_i_x = x_covar
    print(sigma_i_x)
    sigma_i_xd = np.matmul(d_grad, sigma_i_x)
    print(sigma_i_xd)
    sigma_i = np.bmat([[sigma_i_x, np.transpose(sigma_i_xd)], [sigma_i_xd, d_covar]])
    print(sigma_i)
    sigma_x_i_next = np.hstack([A,Bd]).dot(sigma_i).dot(np.transpose(np.hstack([A,Bd])))
    print(sigma_x_i_next)
    return(x_i_next, sigma_x_i_next)

In [20]:
x0

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [21]:
x_gp = [x0[0,:], x0[1,:]]
GP_predict(x_gp)

[array([0.]), array([0.])]
tensor([[0.],
        [0.]], requires_grad=True)


/tmp/ipykernel_15549/1637952563.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  X_grad = torch.autograd.Variable(torch.Tensor(X), requires_grad = True)


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 2 but got size 1 for tensor number 1 in the list.

In [22]:
x = [0,0]
x_ = x[0]
y_ = x[1]
x_gp = [[x_, y_]]
GP_predict(x_gp)

[[0, 0]]
tensor([[0., 0.]], requires_grad=True)
test
test
test
test
test
test


(array([[ 2.27506566e+00],
        [-1.18832235e-04]], dtype=float32),
 array([[0.03416535, 0.        ],
        [0.        , 0.00115217]], dtype=float32),
 array([[-1.86019734e-01,  2.30687292e-05,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-2.35518292e-01,  5.72945146e-06,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]))

In [8]:
def discrete_model(x_mean, x_covar, u):
    # Retrieve disturbance information 
    x_gp = np.transpose(x_mean[0:2,:])
    d_mean, d_covar, d_grad = GP_predict(x_gp)
    print(d_mean)
    # Compute the mean of the next state
    x_full = np.vstack([x_mean, d_mean])
    x_i_next = np.matmul(np.hstack([A_d,B_d_disc]), x_full) + np.matmul(B_d,u)
    # Compute the covariance of the next state 
    sigma_i_x = x_covar
    sigma_i_xd = np.matmul(d_grad, sigma_i_x)
    sigma_i = np.bmat([[sigma_i_x, np.transpose(sigma_i_xd)], [sigma_i_xd, d_covar]])
    stack_mat = np.hstack([A_d,B_d_disc])
    prod1 = np.hstack([A_d,B_d_disc]).dot(sigma_i)
    sigma_x_i_next = np.hstack([A_d,B_d_disc]).dot(sigma_i).dot(np.transpose(np.hstack([A_d,B_d_disc])))
    return(x_i_next, sigma_x_i_next, stack_mat, prod1)

In [9]:
x = [1, 2,3 ,4, 5]
test = (np.array(x))
test.reshape((5,1))

array([[1],
       [2],
       [3],
       [4],
       [5]])

In [10]:
x_i_next[:,0]

NameError: name 'x_i_next' is not defined

In [26]:
data = np.load('/home/johanna/MasterThesis/data/wind_data_1.npz')
data_X = data['X']
data_y = data['y']
data_vel = data['velocity_pred']

KeyError: 'velocity_pred is not a file in the archive'

In [27]:
data.keys()

KeysView(<numpy.lib.npyio.NpzFile object at 0x7f45570cf550>)

In [13]:
import casadi

In [29]:
# Casadi stuff 

x0 = casadi.SX.sym("x_0")
x1 = casadi.SX.sym("x_1")

In [30]:
X = np.array([x0, x1])
casadi.DM(x0.detach().numpy())

Exception: Implicit conversion of symbolic CasADi type to numeric matrix not supported.
This may occur when you pass a CasADi object to a numpy function.
Use an equivalent CasADi function instead of that numpy function.

In [ ]:
X_grad = torch.autograd.Variable(torch.Tensor(X), requires_grad = True)

In [31]:
def GP_predict(X):
    print(X)
    X_grad = torch.autograd.Variable(torch.Tensor(X), requires_grad = True)
    print(X_grad)
    observed_pred1 = (model1(X_grad))
    mean1 = observed_pred1.mean.detach().numpy()
    covar1 = observed_pred1.covariance_matrix.detach().numpy()
    dydtest_x_ag = torch.autograd.grad(observed_pred1.mean.sum(), X_grad)[0]
    y_grad1 = dydtest_x_ag.detach().numpy()
    observed_pred2 = (model2(X_grad))
    mean2 = observed_pred2.mean.detach().numpy()
    covar2 = observed_pred2.covariance_matrix.detach().numpy()
    dydtest_x_ag = torch.autograd.grad(observed_pred2.mean.sum(), X_grad)[0]
    y_grad2 = dydtest_x_ag.detach().numpy()
    y_mean = np.vstack((mean1, mean2))
    y_covar = np.diag([covar1[0,0], covar2[0,0]])
    y_grad = np.zeros([2,9])
    y_grad[:,0] = y_grad1
    y_grad[:,1] = y_grad2
    return y_mean, y_covar, y_grad
    #plot_wind_field(X=X_grid[1300:1301], y=np.transpose(y), res=1)

In [32]:
from casadi import * 

In [33]:
class GPR(Callback):
    def __init__(self, name, opts: dict = None):
        if opts is None:
            opts = dict()

        Callback.__init__(self)
        self.construct(name, opts)
        
    def eval(self, arg):
        X_grad = torch.autograd.Variable(np.array([arg[0], arg[1]]), requires_grad = True)
        observed_pred1 = (model1(X_grad))
        mean1 = observed_pred1.mean.detach().numpy()
        covar1 = observed_pred1.covariance_matrix.detach().numpy()
        dydtest_x_ag = torch.autograd.grad(observed_pred1.mean.sum(), X_grad)[0]
        y_grad1 = dydtest_x_ag.detach().numpy()
        observed_pred2 = (model2(X_grad))
        mean2 = observed_pred2.mean.detach().numpy()
        covar2 = observed_pred2.covariance_matrix.detach().numpy()
        dydtest_x_ag = torch.autograd.grad(observed_pred2.mean.sum(), X_grad)[0]
        y_grad2 = dydtest_x_ag.detach().numpy()
        y_mean = np.vstack((mean1, mean2))
        y_covar = np.diag([covar1[0,0], covar2[0,0]])
        y_grad = np.zeros([2,9])
        y_grad[:,0] = y_grad1
        y_grad[:,1] = y_grad2
        return y_mean

In [34]:
gpr = GPR('GPR')
print(gpr)

GPR:(i0)->(o0) CallbackInternal


In [35]:
w = casadi.SX.sym('w', 2)
DM(w)

DM([nan, nan])

In [36]:
prob = {'f': gpr(w), 'x': w, }
print(prob)

RuntimeError: .../casadi/core/function_internal.cpp:1821: 'eval_sx' not defined for CallbackInternal

In [37]:
from casadi import *
import tensorflow as tf
import gpflow

# Needs: pip install gpflow casadi

# Create data points: a noisy sine wave
N = 20
np.random.seed(0)
data = np.linspace(0, 10, N).reshape((N, 1))
value = np.sin(data) + np.random.normal(0, 0.1, (N, 1))

# Perform Gaussian process regression
model = gpflow.models.GPR((data, value), gpflow.kernels.Constant(1) + gpflow.kernels.Linear(1) + gpflow.kernels.White(
    1) + gpflow.kernels.RBF(1))
opt = gpflow.optimizers.Scipy()
opt.minimize(model.training_loss, model.trainable_variables, options=dict(maxiter=100))


# Package the resulting regression model in a CasADi callback
class GPR(Callback):
    def __init__(self, name, opts: dict = None):
        if opts is None:
            opts = dict()

        Callback.__init__(self)
        self.construct(name, opts)

    def eval(self, arg):
        [mean, _] = model.predict_f(np.array(arg[0]))
        return [mean]


# Instantiate the Callback (make sure to keep a reference to it!)
gpr = GPR('GPR', opts={"enable_fd": True})
print(gpr)

# Find the minimum of the regression model
x = MX.sym("x")
solver = nlpsol("solver", "ipopt", {"x": x, "f": gpr(x)})
res = solver(x0=5)

plot(res["x"], gpr(res["x"]), 'k*', markersize=10, label="Function minimum by CasADi/Ipopt")
legend()
savefig('gpflow1d_min.png', bbox_inches='tight')

ModuleNotFoundError: No module named 'tensorflow'

# Sparse GP hard code

In [9]:
model = GPModel(inducing_points=inducing_points)
state_dict = torch.load('gpytorch_y1_model.pth')
model.load_state_dict(state_dict)
print(state_dict)

constraint = model.covar_module.raw_outputscale_constraint
length_scale = constraint.transform(state_dict['covar_module.base_kernel.raw_lengthscale']).detach().numpy()[0]
output_scale = constraint.transform(state_dict['covar_module.raw_outputscale']).detach().numpy()
u = state_dict['variational_strategy.inducing_points'].detach().numpy()
mu = np.array([state_dict['variational_strategy._variational_distribution.variational_mean'].detach().numpy()])
S = state_dict['variational_strategy._variational_distribution.chol_variational_covar'].detach().numpy()

OrderedDict([('variational_strategy.inducing_points', tensor([[  0.6238,   1.6735],
        [  2.4993,  -1.8205],
        [  8.2422,  -0.1356],
        [  7.5675,   6.6363],
        [  2.2974,   9.2219],
        [ -6.2836,   9.3662],
        [ -5.9367,   1.5368],
        [ -4.8024,  -1.7412],
        [ -4.5350,  -3.1451],
        [ -8.9693,  -8.3928],
        [ -5.7966, -10.0445],
        [ -0.3767,  -9.9683],
        [  4.4215,  -9.5435],
        [  8.5375,  -9.7206],
        [  3.6561,  -5.3003],
        [ -1.5771,  -0.5819],
        [ -9.0627,  -1.9239],
        [ -9.8440,   0.3330],
        [ -9.4903,   3.4844],
        [ -8.9786,   6.7935],
        [ -2.4628,   5.3270],
        [  8.4513,   9.5683],
        [  9.6003,   3.7080],
        [  9.3990,  -3.3051],
        [  9.6274,  -5.7380],
        [  5.7499,   2.5925],
        [  1.9121,   4.7352],
        [ -8.7800,  -3.0534],
        [ -5.1636,  -4.1952],
        [ -2.0399,  -6.6442],
        [  6.2088,  -2.2447]])), ('variational

In [10]:
from scipy.spatial.distance import pdist, cdist, squareform
import casadi as cs

In [28]:
class GPpredict_():
    def __init__(self, l, sigma_f, u, mu, S):
        self.l=l
        self.sigma_f=sigma_f
        self.u=u
        self.mu=mu
        self.S=S
        self.Kuu = self.squared_exponential_kernel(u, u) + 1e-03* np.identity(self.u.shape[-2])
        self.inv_L = np.linalg.inv(np.linalg.cholesky(self.Kuu))
        self.var = np.matmul(S, np.transpose(S)) - np.identity(S.shape[-1])
    def squared_exponential_kernel(self, x_1, x_2=None):
        """
        Anisotropic (diagonal length-scale) matrix squared exponential kernel. Computes a covariance matrix from points
        in x_1 and x_2.

        Args:
            x_1: Array of m points (m x d).
            x_2: Array of n points (n x d).

        Returns:
            Covariance matrix (m x n).
        """

        if isinstance(x_2, cs.SX):
            return self._squared_exponential_kernel_cs(x_1, x_2)

        # Length scale parameter
        len_scale = self.l

        # Vertical variation parameter
        sigma_f = self.sigma_f

        x_1 = np.atleast_2d(x_1)
        length_scale = self._check_length_scale(x_1, len_scale)
        if x_2 is None:
            dists = pdist(x_1 / length_scale, metric='sqeuclidean')
            k = sigma_f * np.exp(-.5 * dists)
            # convert from upper-triangular matrix to square matrix
            k = squareform(k)
            np.fill_diagonal(k, 1)
        else:
            dists = cdist(x_1 / length_scale, x_2 / length_scale, metric='sqeuclidean')
            k = sigma_f * np.exp(-.5 * dists)

        return k
    
    def _check_length_scale(self, x, length_scale):
        length_scale = np.squeeze(length_scale).astype(float)
        if np.ndim(length_scale) > 1:
            raise ValueError("length_scale cannot be of dimension greater than 1")
        if np.ndim(length_scale) == 1 and x.shape[1] != length_scale.shape[0]:
            raise ValueError("Anisotropic kernel must have the same number of dimensions as data (%d!=%d)"
                             % (length_scale.shape[0], x.shape[1]))
        return length_scale

    def _squared_exponential_kernel_cs(self, x_1, x_2):
        """
        Symbolic implementation of the anisotropic squared exponential kernel
        :param x_1: Array of m points (m x d).
        :param x_2: Array of n points (m x d).
        :return: Covariance matrix (m x n).
        """

        # Length scale parameter
        len_scale = self.l
        # Vertical variation parameter
        sigma_f =  self.sigma_f

        if x_1.shape != x_2.shape and x_2.shape[0] == 1:
            tiling_ones = cs.SX.ones(x_1.shape[0], 1)
            d = x_1 - cs.mtimes(tiling_ones, x_2)
            dist = cs.sum2(d ** 2 / cs.mtimes(tiling_ones, cs.SX(len_scale ** 2).T))
        else:
            d = x_1 - x_2
            dist = cs.sum1(d ** 2 / cs.SX(len_scale ** 2).T)

        return sigma_f * cs.SX.exp(-.5 * dist)
    def predict(self, X):
        Kxu = self.squared_exponential_kernel(self.u, X)
        Kxx = self.squared_exponential_kernel(X, X) + 1e-04* np.identity(X.shape[-2])
        interp_term = cs.mtimes(self.inv_L, Kxu)
        mean = cs.mtimes(self.mu, interp_term)
        covar = Kxx + cs.mtimes(interp_term.T,cs.mtimes(self.var, interp_term))
        return mean, covar
    def mean(self, X):
        Kxu = self.squared_exponential_kernel(self.u, X)
        interp_term = cs.mtimes(self.inv_L, Kxu)
        mean = cs.mtimes(self.mu, interp_term)
        return mean
    def covar(self, X):
        Kxu = self.squared_exponential_kernel(self.u, X)
        Kxx = self.squared_exponential_kernel(X, X) + 1e-04* np.identity(X.shape[-2])
        interp_term = cs.mtimes(self.inv_L, Kxu)
        covar = Kxx + cs.mtimes(interp_term.T,cs.mtimes(self.var, interp_term))
        return mean

In [29]:
x1 = cs.SX.sym('x_5', 1)
x2 = cs.SX.sym('x_6', 1)
print(x1,x2)
X = cs.horzcat(x1,x2)
print(X)

x_5 x_6
[[x_5, x_6]]


In [30]:
def f(x1, x2):
    X = cs.horzcat(x1,x2)
    mean= predict.mean(X)
    return mean

In [31]:
X = np.array([[-9,0]])

In [32]:
f1 = cs.Function('f',[x1,x2],[f(x1,x2)]);
f1(-9,0)

NameError: name 'predict' is not defined

In [33]:
predict = GPpredict_(length_scale, output_scale, u, mu, S)
mean= predict.mean(X)
print(mean)

NameError: name 'length_scale' is not defined

In [20]:
from casadi import *

In [21]:
#Load the trained GP model
#Load the data
data = np.load('/home/johanna/MasterThesis/data/wind_data_4.npz')
data_X = data['X']
data_y = data['y']
inducing_points = torch.FloatTensor(data_X[::int(data_y.shape[0]/30), :])

#Load model in first direction
model1 = GPModel(inducing_points=inducing_points)
state_dict = torch.load('/home/johanna/MasterThesis/Pyhton/gpytorch_y1_model.pth')
model1.load_state_dict(state_dict)
constraint1 = model1.covar_module.raw_outputscale_constraint
length_scale1 = constraint1.transform(state_dict['covar_module.base_kernel.raw_lengthscale']).detach().numpy()[0]
output_scale1 = constraint1.transform(state_dict['covar_module.raw_outputscale']).detach().numpy()
u1 = state_dict['variational_strategy.inducing_points'].detach().numpy()
mu1 = np.array([state_dict['variational_strategy._variational_distribution.variational_mean'].detach().numpy()])
S1 = state_dict['variational_strategy._variational_distribution.chol_variational_covar'].detach().numpy()

prediction_model1 = GPpredict_(length_scale1, output_scale1, u1, mu1, S1)

model2 = GPModel(inducing_points=inducing_points)
state_dict = torch.load('/home/johanna/MasterThesis/Pyhton/gpytorch_y2_model.pth')
model2.load_state_dict(state_dict)
constraint2 = model2.covar_module.raw_outputscale_constraint
length_scale2 = constraint2.transform(state_dict['covar_module.base_kernel.raw_lengthscale']).detach().numpy()[0]
output_scale2 = constraint2.transform(state_dict['covar_module.raw_outputscale']).detach().numpy()
u2 = state_dict['variational_strategy.inducing_points'].detach().numpy()
mu2 = np.array(
    [state_dict['variational_strategy._variational_distribution.variational_mean'].detach().numpy()])
S2 = state_dict['variational_strategy._variational_distribution.chol_variational_covar'].detach().numpy()

prediction_model2 = GPpredict_(length_scale2, output_scale2, u2, mu2, S2)

In [22]:
def f(x1, x2):
    X = cs.horzcat(x1,x2)
    mean1= prediction_model1.mean(X)
    return mean1
f1 = cs.Function('f',[x1,x2],[f(x1,x2)]);
f1(-9,0)

NameError: name 'x1' is not defined

In [132]:
class GPpredict_():
    def __init__(self, l, sigma_f, u, mu, S):
        self.l = l
        self.sigma_f = sigma_f
        self.u = u
        self.mu = mu
        self.S = S
        self.Kuu = self.squared_exponential_kernel(u, u) + 1e-03 * np.identity(self.u.shape[-2])
        self.inv_L = np.linalg.inv(np.linalg.cholesky(self.Kuu))
        self.var = np.matmul(S, np.transpose(S)) - np.identity(S.shape[-1])

    def squared_exponential_kernel(self, x_1, x_2=None):
        """
        Anisotropic (diagonal length-scale) matrix squared exponential kernel. Computes a covariance matrix from points
        in x_1 and x_2.

        Args:
            x_1: Array of m points (m x d).
            x_2: Array of n points (n x d).

        Returns:
            Covariance matrix (m x n).
        """

        if isinstance(x_2, casadi.SX):
            return self._squared_exponential_kernel_cs(x_1, x_2)

        # Length scale parameter
        len_scale = self.l

        # Vertical variation parameter
        sigma_f = self.sigma_f

        x_1 = np.atleast_2d(x_1)
        length_scale = self._check_length_scale(x_1, len_scale)
        if x_2 is None:
            dists = pdist(x_1 / length_scale, metric='sqeuclidean')
            k = sigma_f * np.exp(-.5 * dists)
            # convert from upper-triangular matrix to square matrix
            k = squareform(k)
            np.fill_diagonal(k, 1)
        else:
            dists = cdist(x_1 / length_scale, x_2 / length_scale, metric='sqeuclidean')
            k = sigma_f * np.exp(-.5 * dists)

        return k

    def _check_length_scale(self, x, length_scale):
        length_scale = np.squeeze(length_scale).astype(float)
        if np.ndim(length_scale) > 1:
            raise ValueError("length_scale cannot be of dimension greater than 1")
        if np.ndim(length_scale) == 1 and x.shape[1] != length_scale.shape[0]:
            raise ValueError("Anisotropic kernel must have the same number of dimensions as data (%d!=%d)"
                             % (length_scale.shape[0], x.shape[1]))
        return length_scale

    def _squared_exponential_kernel_cs(self, x_1, x_2):
        """
        Symbolic implementation of the anisotropic squared exponential kernel
        :param x_1: Array of m points (m x d).
        :param x_2: Array of n points (m x d).
        :return: Covariance matrix (m x n).
        """

        # Length scale parameter
        len_scale = self.l
        # Vertical variation parameter
        sigma_f = self.sigma_f

        if x_1.shape != x_2.shape and x_2.shape[0] == 1:
            tiling_ones = casadi.SX.ones(x_1.shape[0], 1)
            d = x_1 - casadi.mtimes(tiling_ones, x_2)
            dist = casadi.sum2(d ** 2 / casadi.mtimes(tiling_ones, casadi.SX(len_scale ** 2).T))
        else:
            d = x_1 - x_2
            dist = casadi.sum1(d ** 2 / casadi.SX(len_scale ** 2).T)

        return sigma_f * casadi.SX.exp(-.5 * dist)

    def predict(self, X):
        Kxu = self.squared_exponential_kernel(self.u, X)
        Kxx = self.squared_exponential_kernel(X, X) + 1e-04 * np.identity(X.shape[-2])
        interp_term = casadi.mtimes(self.inv_L, Kxu)
        mean = casadi.mtimes(self.mu, interp_term)
        covar = Kxx + casadi.mtimes(interp_term.T, casadi.mtimes(self.var, interp_term))
        return mean, covar

    def mean(self, X):
        Kxu = self.squared_exponential_kernel(self.u, X)
        interp_term = casadi.mtimes(self.inv_L, Kxu)
        test = casadi.mtimes(self.mu, self.inv_L)
        print(test.shape)
        mean = casadi.mtimes(self.mu, interp_term)
        return mean

    def covar(self, X):
        Kxu = self.squared_exponential_kernel(self.u, X)
        Kxx = self.squared_exponential_kernel(X, X) + 1e-04 * np.identity(X.shape[-2])
        interp_term = casadi.mtimes(self.inv_L, Kxu)
        print(interp_term.shape)
        covar = Kxx + casadi.mtimes(interp_term.T, casadi.mtimes(self.var, interp_term))
        return covar

class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [18]:
def _matern_kernel_cs(x_1, x_2, length_scale):
        """
        Symbolic implementation of the anisotropic squared exponential kernel
        :param x_1: Array of m points (m x d).
        :param x_2: Array of n points (m x d).
        :return: Covariance matrix (m x n).
        """

        # Length scale parameter
        len_scale = length_scale

        d = (abs(x_1[0]-x_2[0]))/len_scale[0] **2+ (abs(x_1[1]-x_2[1]))/len_scale[1] # TODO: check if squared or not
        print(d) 
        k = 1 + sqrt(5) * d + 5/3 * d ** 2 + casadi.SX.exp(- sqrt(5) * d)

        return k

In [20]:
def cov_matrix(x1, x2, cov_function, length_scale, output_scale) -> np.array:
    return np.array([[cov_function(a, b, length_scale, output_scale) for a in x1] for b in x2])

In [369]:
import torch
import gpytorch
from torch.utils.data import TensorDataset, DataLoader
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

In [89]:
covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2))
covar_module.outputscale = output_scale
covar_module.base_kernel.lengthscale = torch.Tensor(length_scale)

In [370]:
x1 = torch.randn(10, 2)
x2 = torch.randn(10, 2)
covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=2))
covar = covar_module(x1,x2)  # Output: LazyVariable of size (10 x 10)
print(covar.detach().numpy())

[[2.33055912e-02 2.08357006e-01 4.43746150e-02 5.62368147e-03
  2.39281014e-01 2.71179438e-01 4.87431675e-01 6.59181893e-01
  4.86321412e-02 2.86762923e-01]
 [1.60200074e-02 9.05320805e-04 2.14634649e-02 1.60669163e-02
  8.49599659e-04 1.23536796e-03 7.37863884e-04 4.27832507e-04
  2.00892910e-02 1.47261890e-04]
 [1.16611786e-01 2.01199815e-01 3.26504976e-01 3.97158824e-02
  2.16114700e-01 3.98408860e-01 3.47797453e-01 2.90645540e-01
  3.50367129e-01 1.07362427e-01]
 [1.91233680e-01 5.90712018e-02 6.15330637e-01 9.88800302e-02
  5.94249405e-02 1.09675847e-01 7.11840019e-02 4.90280092e-02
  6.19557202e-01 1.80834476e-02]
 [3.43802082e-03 3.90245090e-03 7.52145946e-02 2.49917284e-01
  4.42508003e-03 1.08670682e-01 1.23194708e-02 1.70405824e-02
  7.09801465e-02 4.88254838e-02]
 [7.22265523e-03 4.73513119e-02 7.76290544e-04 3.13388009e-05
  4.36096936e-02 1.50620076e-03 1.71983484e-02 1.06113255e-02
  8.86396912e-04 1.20136410e-03]
 [4.41306174e-01 1.80603996e-01 4.01793607e-02 2.09435821e

# Check Dynamics Model

In [34]:
from casadi import *
import torch
from scipy.spatial.distance import pdist, cdist, squareform
import casadi as cs
import gpytorch
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

In [35]:
class DynamicModel:

    def __init__(self, system):
        self.nvar = self.nu + self.nx
        self.system = system

    def __str__(self):
        result = 'Dynamical Model: ' + str(type(self)) + '\n' +\
               'System: ' + str(self.system) + '\n'

        if hasattr(self, 'interfaces'):
            result += 'Interfaces: '

            for interface in self.interfaces:
                result += interface + " "

            result += "\n"

        result += 'States: ' + str(self.states) + '\n'
        result += 'Inputs: ' + str(self.inputs) + '\n'
        return result

    # Appends upper bounds from system
    def upper_bound(self):
        result = np.array([])

        for input in self.inputs:
            result = np.append(result, self.system.upper_bound[input])

        for state in self.states:
            result = np.append(result, self.system.upper_bound[state])

        return result

    # Appends lower bounds from system
    def lower_bound(self):
        result = np.array([])

        for input in self.inputs:
            result = np.append(result, self.system.lower_bound[input])

        for state in self.states:
            result = np.append(result, self.system.lower_bound[state])

        return result


In [36]:
class DroneModel(DynamicModel):

    def __init__(self, system, options):
        # Check whether model should be nonlinear and whether yaw component should be included or not
        self.nonlin = options['nonlin']
        self.with_yaw = options['with_yaw']

        if not self.with_yaw:
            self.states = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'phi', 'theta', 'v', 'spline']
            self.states_from_sensor = [True, True, True, True, True, True, True, True, True, False]
            self.states_from_sensor_at_infeasible = [True, True, True, True, True, True, True, True, True, False]

            self.inputs = ['phi_c', 'theta_c', 'thrust_c', 'a']
            self.possible_inputs_to_vehicle = ['phi_c', 'theta_c', 'thrust_c', 'a']
        else:
            self.states = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'phi', 'theta', 'psi', 'v', 'spline']
            self.states_from_sensor = [True, True, True, True, True, True, True, True, True, True, False]
            self.states_from_sensor_at_infeasible = [True, True, True, True, True, True, True, True, True, True, False]

            self.inputs = ['phi_c', 'theta_c', 'dpsi_c', 'thrust_c', 'a']
            self.possible_inputs_to_vehicle = ['phi_c', 'theta_c', 'dpsi_c', 'thrust_c', 'a']

        # Load the trained GP model
        # Load the data
        data = np.load('/home/johanna/MasterThesis/data/wind_data_4.npz')
        data_X = data['X']
        data_y = data['y']
        inducing_points = torch.FloatTensor(data_X[::int(data_y.shape[0] / 30), :])

        # Load model in first direction
        self.model1 = GPModel(inducing_points=inducing_points)
        state_dict = torch.load('/home/johanna/MasterThesis/Pyhton/gpytorch_y1_model.pth')
        self.model1.load_state_dict(state_dict)
        constraint1 = self.model1.covar_module.raw_outputscale_constraint
        length_scale1 = constraint1.transform(state_dict['covar_module.base_kernel.raw_lengthscale']).detach().numpy()[0]
        output_scale1 = constraint1.transform(state_dict['covar_module.raw_outputscale']).detach().numpy()
        u1 = state_dict['variational_strategy.inducing_points'].detach().numpy()
        mu1 = np.array(
            [state_dict['variational_strategy._variational_distribution.variational_mean'].detach().numpy()])
        S1 = state_dict['variational_strategy._variational_distribution.chol_variational_covar'].detach().numpy()

        self.prediction_model1 = GPpredict_(length_scale1, output_scale1, u1, mu1, S1)

        self.model2 = GPModel(inducing_points=inducing_points)
        state_dict = torch.load('/home/johanna/MasterThesis/Pyhton/gpytorch_y2_model.pth')
        self.model2.load_state_dict(state_dict)
        constraint2 = self.model2.covar_module.raw_outputscale_constraint
        length_scale2 = constraint2.transform(state_dict['covar_module.base_kernel.raw_lengthscale']).detach().numpy()[0]
        output_scale2 = constraint2.transform(state_dict['covar_module.raw_outputscale']).detach().numpy()
        u2 = state_dict['variational_strategy.inducing_points'].detach().numpy()
        mu2 = np.array(
            [state_dict['variational_strategy._variational_distribution.variational_mean'].detach().numpy()])
        S2 = state_dict['variational_strategy._variational_distribution.chol_variational_covar'].detach().numpy()

        self.prediction_model2 = GPpredict_(length_scale2, output_scale2, u2, mu2, S2)

        self.nu = len(self.inputs)
        self.nx = len(self.states)
        super(DroneModel, self).__init__(system)

    def continuous_model(self, x, u):
        # Load runtime parameters
        # settings.weights.set_weights(param)

        # Input to the GP
        x_gp = casadi.horzcat(x[0], x[1])

        # States
        vx = x[3]
        vy = x[4]
        vz = x[5]
        phi = x[6]
        theta = x[7]
        if not self.with_yaw:
            v = x[8]
            spline = x[9]
        else:
            psi = x[8]
            v = x[9]
            spline = x[10]

        # Inputs
        phi_c = u[0]            # Commanded roll angle around body frame x-axis [rad]
        theta_c = u[1]          # Commanded pitch angle around body frame y-axis [rad]
        if not self.with_yaw:
            thrust_c = u[2]     # Commanded acceleration along body frame z-axis [m/s^2]
            a = u[3]
        else:
            dpsi_c = u[2]       # Commanded yaw rate around body frame z-axis [rad/s]
            thrust_c = u[3]     # Commanded acceleration along body frame z-axis [m/s^2]
            a = u[4]
        # Note: thrust_c = desired thrust (N) / mass (kg)
        # The final commanded thrust to PX4 is T_c in [0, 1]
        # Find out relation between thrust_c and T_c by:
        # - Hovering experiments with different mass: (m+delta_m)*g = T_c' = m*(g+thrust_c)
        # - Measuring PWM-thrust relation with force sensor: [0, 1] maps linearly to PWM values
        # Note also: a is a virtual control input, also called progress acceleration, allowing to limit the reference
        # velocity along the path when v is not included in the standard system states, and therefore included as an
        # extra state v

        # Identified constants
        k_phi = 1.04
        tau_phi = 0.17
        k_theta = 1.04
        tau_theta = 0.17
        # kD_x = 0.02  # TODO: minimum value from ETH paper
        # kD_y = 0.02  # TODO: minimum value from ETH paper

        # Constants
        m = 1.9
        g = 9.81

        # Calculate derivatives
        dx = vx
        dy = vy
        dz = vz

        if not self.nonlin:
            dvx = g * theta
            dvy = -g * phi
            # dvx = -kD_x * vx + g * theta
            # dvy = -kD_y * vy - g * phi
            dvz = thrust_c
        else:
            dvx = thrust_c * (casadi.sin(phi) * casadi.sin(psi) + casadi.cos(phi) * casadi.sin(theta) * casadi.cos(psi))
            dvy = thrust_c * (-casadi.sin(phi) * casadi.cos(psi) + casadi.cos(phi) * casadi.sin(theta) * casadi.sin(psi))
            dvz = thrust_c * (casadi.cos(phi) * casadi.cos(theta)) - g
        dphi = (k_phi * phi_c - phi) / tau_phi
        dtheta = (k_theta * theta_c - theta) / tau_theta
        if self.with_yaw:
            ddpsi = dpsi_c
        # dspline = vy  # TODO Change this value to something else
        # v = casadi.norm_2(np.array([vx, vy]))
        # v = casadi.norm_2(np.array([vx, vy, vz]))
        # dspline = v
        # dspline = casadi.sqrt(2)*vx
        # dspline = settings.weights.velocity_reference
        dv = a
        dspline = v

        if not self.with_yaw:
            return np.array([dx, dy, dz, dvx, dvy, dvz, dphi, dtheta, dv, dspline])
        else:
            return np.array([dx, dy, dz, dvx, dvy, dvz, dphi, dtheta, ddpsi, dv, dspline])

In [105]:
# Linear Gaussian Process Model
class DroneGPModel(DynamicModel):

    def __init__(self, system, options):
        # Check whether model should be nonlinear and whether yaw component should be included or not
        self.nonlin = options['nonlin']
        self.with_yaw = options['with_yaw']

        if not self.with_yaw:
            self.states = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'phi', 'theta', 'v', 'spline']
            self.states_from_sensor = [True, True, True, True, True, True, True, True, True, False]
            self.states_from_sensor_at_infeasible = [True, True, True, True, True, True, True, True, True, False]

            self.inputs = ['phi_c', 'theta_c', 'thrust_c', 'a']
            self.possible_inputs_to_vehicle = ['phi_c', 'theta_c', 'thrust_c', 'a']
        else:
            self.states = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'phi', 'theta', 'psi', 'v', 'spline']
            self.states_from_sensor = [True, True, True, True, True, True, True, True, True, True, False]
            self.states_from_sensor_at_infeasible = [True, True, True, True, True, True, True, True, True, True, False]

            self.inputs = ['phi_c', 'theta_c', 'dpsi_c', 'thrust_c', 'a']
            self.possible_inputs_to_vehicle = ['phi_c', 'theta_c', 'dpsi_c', 'thrust_c', 'a']

        # Constants
        self.g = 9.81

        # Identified constants
        self.tau_phi = 0.17
        self.k_phi = 1.04
        self.tau_theta = 0.17
        self.k_theta = 1.04
        self.kD = 0.02  # Drag coefficient

        # Initialize continuous matrices
        if not self.with_yaw:
            self.A = np.zeros([8,8])
            self.A[0,3] = 1
            self.A[1,4] = 1
            self.A[2,5] = 1
            self.A[3,7] = self.g
            self.A[4,6] = -self.g
            self.A[6,6] = -1/self.tau_phi
            self.A[7,7] = -1/self.tau_theta
            self.B = np.zeros([8,3])
            self.B[6,0] = self.k_theta/self.tau_theta
            self.B[7,1] = self.k_theta/self.tau_phi
            self.Bd = np.zeros([8,2])
            self.Bd[3,0] = 1
            self.Bd[4,1] = 1
        else:
            self.A = np.zeros([9,9])
            self.A[0,3] = 1
            self.A[1,4] = 1
            self.A[2,5] = 1
            self.A[3,7] = self.g
            self.A[4,6] = -self.g
            self.A[6,6] = -1/self.tau_phi
            self.A[7,7] = -1/self.tau_theta
            self.B = np.zeros([9,4])
            self.B[5,3] = 1
            self.B[6,0] = self.k_theta/self.tau_theta
            self.B[7,1] = self.k_theta/self.tau_phi
            self.B[8,2] = 1
            self.Bd = np.zeros([9,2])
            self.Bd[3,0] = 1
            self.Bd[4,1] = 1
            
        print(self.A, self.B)

        #Load the trained GP model
        #Load the data
        data = np.load('/home/johanna/MasterThesis/data/wind_data_4.npz')
        data_X = data['X']
        data_y = data['y']
        inducing_points = torch.FloatTensor(data_X[::int(data_y.shape[0]/30), :])

        #Load model in first direction
        self.model1 = GPModel(inducing_points=inducing_points)
        state_dict = torch.load('/home/johanna/MasterThesis/Pyhton/gpytorch_y1_model.pth')
        self.model1.load_state_dict(state_dict)
        constraint1 = self.model1.covar_module.raw_outputscale_constraint
        length_scale1 = constraint1.transform(state_dict['covar_module.base_kernel.raw_lengthscale']).detach().numpy()[0]
        output_scale1 = constraint1.transform(state_dict['covar_module.raw_outputscale']).detach().numpy()
        u1 = state_dict['variational_strategy.inducing_points'].detach().numpy()
        mu1 = np.array([state_dict['variational_strategy._variational_distribution.variational_mean'].detach().numpy()])
        S1 = state_dict['variational_strategy._variational_distribution.chol_variational_covar'].detach().numpy()

        self.prediction_model1 = GPpredict_(length_scale1, output_scale1, u1, mu1, S1)

        self.model2 = GPModel(inducing_points=inducing_points)
        state_dict = torch.load('/home/johanna/MasterThesis/Pyhton/gpytorch_y2_model.pth')
        self.model2.load_state_dict(state_dict)
        constraint2 = self.model2.covar_module.raw_outputscale_constraint
        length_scale2 = constraint2.transform(state_dict['covar_module.base_kernel.raw_lengthscale']).detach().numpy()[0]
        output_scale2 = constraint2.transform(state_dict['covar_module.raw_outputscale']).detach().numpy()
        u2 = state_dict['variational_strategy.inducing_points'].detach().numpy()
        mu2 = np.array(
            [state_dict['variational_strategy._variational_distribution.variational_mean'].detach().numpy()])
        S2 = state_dict['variational_strategy._variational_distribution.chol_variational_covar'].detach().numpy()

        self.prediction_model2 = GPpredict_(length_scale2, output_scale2, u2, mu2, S2)

        self.nu = len(self.inputs)
        self.nx = len(self.states)
        super(DroneGPModel, self).__init__(system)

    def continuous_model(self, x, u): #, param, settings):
        # Load runtime parameters
        #settings.weights.set_weights(param)

        # States
        if not self.with_yaw:
            v = x[8]
            spline = x[9]
            x_mean = x[:8]
            inp = u[:3]
        else:
            print("this")
            v = x[9]
            spline = x[10]
            x_mean = x[:9]
            inp = u[:4]

        x_gp = casadi.horzcat(x_mean[0], x_mean[1])
        print(x_gp)
        mean1 = self.prediction_model1.mean(x_gp)
        mean2 = self.prediction_model2.mean(x_gp)
        d_mean = casadi.vertcat(mean1, mean2)
        print(x_mean.shape)
        print(d_mean.shape)
        # d_mean = [test(x_gp[0], x_gp[1]),  test(x_gp[0], x_gp[1])]
        # print(d_mean)

        # Compute the mean of the next state
        x_full = casadi.vertcat(x_mean, d_mean)
        x_i_next = casadi.mtimes(np.hstack([self.A,self.Bd]), x_full) + casadi.mtimes(self.B, inp)

        # Other inputs
        if not self.with_yaw:
            a = u[3]
        else:
            a = u[4]
        # Note: thrust_c = desired thrust (N) / mass (kg)
        # The final commanded thrust to PX4 is T_c in [0, 1]
        # Find out relation between thrust_c and T_c by:
        # - Hovering experiments with different mass: (m+delta_m)*g = T_c' = m*(g+thrust_c)
        # - Measuring PWM-thrust relation with force sensor: [0, 1] maps linearly to PWM values
        # Note also: a is a virtual control input, also called progress acceleration, allowing to limit the reference
        # velocity along the path when v is not included in the standard system states, and therefore included as an
        # extra state v


        # dspline = vy  # TODO Change this value to something else
        # v = casadi.norm_2(np.array([vx, vy]))
        # v = casadi.norm_2(np.array([vx, vy, vz]))
        # dspline = v
        # dspline = casadi.sqrt(2)*vx
        # dspline = settings.weights.velocity_reference
        dv = a
        dspline = v

        if not self.with_yaw:
            return np.array([x_i_next[0], x_i_next[1], x_i_next[2], x_i_next[3], x_i_next[4], x_i_next[5], x_i_next[6], x_i_next[7], dv, dspline])
        else:
            return np.array([x_i_next[0], x_i_next[1], x_i_next[2], x_i_next[3], x_i_next[4], x_i_next[5], x_i_next[6], x_i_next[7], x_i_next[8], dv, dspline])


class GPpredict_():
    def __init__(self, l, sigma_f, u, mu, S):
        self.l = l
        self.sigma_f = sigma_f
        self.u = u
        self.mu = mu
        self.S = S
        self.Kuu = self.squared_exponential_kernel(u, u) + 1e-03 * np.identity(self.u.shape[-2])
        self.inv_L = np.linalg.inv(np.linalg.cholesky(self.Kuu))
        self.var = np.matmul(S, np.transpose(S)) - np.identity(S.shape[-1])

    def squared_exponential_kernel(self, x_1, x_2=None):
        """
        Anisotropic (diagonal length-scale) matrix squared exponential kernel. Computes a covariance matrix from points
        in x_1 and x_2.

        Args:
            x_1: Array of m points (m x d).
            x_2: Array of n points (n x d).

        Returns:
            Covariance matrix (m x n).
        """

        if isinstance(x_2, casadi.SX):
            return self._squared_exponential_kernel_cs(x_1, x_2)

        # Length scale parameter
        len_scale = self.l

        # Vertical variation parameter
        sigma_f = self.sigma_f

        x_1 = np.atleast_2d(x_1)
        length_scale = self._check_length_scale(x_1, len_scale)
        if x_2 is None:
            dists = pdist(x_1 / length_scale, metric='sqeuclidean')
            k = sigma_f * np.exp(-.5 * dists)
            # convert from upper-triangular matrix to square matrix
            k = squareform(k)
            np.fill_diagonal(k, 1)
        else:
            dists = cdist(x_1 / length_scale , x_2 / length_scale, metric='sqeuclidean')
            k = sigma_f * np.exp(-.5 * dists)

        return k

    def _check_length_scale(self, x, length_scale):
        length_scale = np.squeeze(length_scale).astype(float)
        if np.ndim(length_scale) > 1:
            raise ValueError("length_scale cannot be of dimension greater than 1")
        if np.ndim(length_scale) == 1 and x.shape[1] != length_scale.shape[0]:
            raise ValueError("Anisotropic kernel must have the same number of dimensions as data (%d!=%d)"
                             % (length_scale.shape[0], x.shape[1]))
        return length_scale

    def _squared_exponential_kernel_cs(self, x_1, x_2):
        """
        Symbolic implementation of the anisotropic squared exponential kernel
        :param x_1: Array of m points (m x d).
        :param x_2: Array of n points (m x d).
        :return: Covariance matrix (m x n).
        """

        # Length scale parameter
        len_scale = self.l
        # Vertical variation parameter
        sigma_f = self.sigma_f

        if x_1.shape != x_2.shape and x_2.shape[0] == 1:
            tiling_ones = casadi.SX.ones(x_1.shape[0], 1)
            d = x_1 - casadi.mtimes(tiling_ones, x_2)
            dist = casadi.sum2(d ** 2 / casadi.mtimes(tiling_ones, casadi.SX(len_scale ** 2).T))
        else:
            d = x_1 - x_2
            dist = casadi.sum1(d ** 2 / casadi.SX(len_scale ** 2).T)

        return sigma_f * casadi.SX.exp(-.5 * dist)

    def predict(self, X):
        Kxu = self.squared_exponential_kernel(self.u, X)
        Kxx = self.squared_exponential_kernel(X, X) + 1e-04 * np.identity(X.shape[-2])
        interp_term = casadi.mtimes(self.inv_L, Kxu)
        mean = casadi.mtimes(self.mu, interp_term)
        covar = Kxx + casadi.mtimes(interp_term.T, casadi.mtimes(self.var, interp_term))
        return mean, covar

    def mean(self, X):
        Kxu = self.squared_exponential_kernel(self.u, X)
        interp_term = casadi.mtimes(self.inv_L, Kxu)
        mean = casadi.mtimes(self.mu, interp_term)
        return mean

    def covar(self, X):
        Kxu = self.squared_exponential_kernel(self.u, X)
        Kxx = self.squared_exponential_kernel(X, X) + 1e-04 * np.identity(X.shape[-2])
        interp_term = casadi.mtimes(self.inv_L, Kxu)
        covar = Kxx + casadi.mtimes(interp_term.T, casadi.mtimes(self.var, interp_term))
        return covar

class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [106]:
class Hovergames:

    def __str__(self):
        return self.name

    def __init__(self):
        self.name = 'Hovergames'
        self.lower_bound = dict()
        self.upper_bound = dict()

        # Maximum roll and pitch angle
        max_angle = 50  # deg
        max_angle_cmd = max_angle - 10  # deg

        max_yawrate = 15  # deg/s
        max_yawrate_cmd = max_yawrate - 5  # deg/s

        self.lower_bound['x'] = -2000.0
        self.upper_bound['x'] = 2000.0

        self.lower_bound['y'] = -2000.0
        self.upper_bound['y'] = 2000.0

        self.lower_bound['z'] = -1.0
        self.upper_bound['z'] = 3.0

        self.lower_bound['vx'] = -4.0
        self.upper_bound['vx'] = 4.0

        self.lower_bound['vy'] = -4.0
        self.upper_bound['vy'] = 4.0

        self.lower_bound['vz'] = -1.0
        self.upper_bound['vz'] = 4.0

        self.lower_bound['phi'] = -np.pi / 180 * max_angle
        self.upper_bound['phi'] = np.pi / 180 * max_angle

        self.lower_bound['theta'] = -np.pi / 180 * max_angle
        self.upper_bound['theta'] = np.pi / 180 * max_angle

        self.lower_bound['psi'] = -np.pi / 180 * max_angle
        self.upper_bound['psi'] = np.pi / 180 * max_angle

        self.lower_bound['v'] = -1.0
        self.upper_bound['v'] = 5.0

        self.lower_bound['spline'] = -1.0
        self.upper_bound['spline'] = 10000.0

        self.lower_bound['phi_c'] = -np.pi / 180 * max_angle_cmd
        self.upper_bound['phi_c'] = np.pi / 180 * max_angle_cmd

        self.lower_bound['theta_c'] = -np.pi / 180 * max_angle_cmd
        self.upper_bound['theta_c'] = np.pi / 180 * max_angle_cmd

        self.lower_bound['dpsi_c'] = -np.pi / 180 * max_yawrate_cmd
        self.upper_bound['dpsi_c'] = np.pi / 180 * max_yawrate_cmd

        self.lower_bound['thrust_c'] = 5
        self.upper_bound['thrust_c'] = 15.0

        self.lower_bound['a'] = -1.0
        self.upper_bound['a'] = 5.0


In [107]:
# --- Model options --- #
model_options = dict()
model_options['nonlin'] = True
model_options['with_yaw'] = True
if model_options['nonlin'] and not model_options['with_yaw']:
    exit("Nonlinear model check failed: with_yaw should be set to True in order to use the nonlinear model!"
         " Check hovergames_settings.py")

In [108]:
robot = Hovergames() 
model = DroneModel(system=robot, options=model_options)
modelGP = DroneGPModel(system=robot, options=model_options)

[[ 0.          0.          0.          1.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          1.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          9.81        0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -9.81        0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -5.88235294  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.         -5.88235294  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]] [[0.         0.         0.         0.        ]
 [0.   

In [114]:
x0 = [0,0,0,0,0,0,0,0,0,0,0]
u0 = [0,0,0,0,0]
test = model.continuous_model(w,u0)
print(test)
test = modelGP.continuous_model(w,u0)
print(test)

[SX(w_3) SX(w_4) SX(w_5) SX(0) SX(0) SX(-9.81) SX((-(w_6/0.17)))
 SX((-(w_7/0.17))) 0 0 SX(w_9)]
this
[[w_0, w_1]]
(9, 1)
(2, 1)
[SX(w_3) SX(w_4) SX(w_5)
 SX(@1=0.30591, @2=-0.5, @3=30.9339, @4=4.78953, @5=(@1*exp((@2*((sq((0.623751-w_0))/@3)+(sq((1.67351-w_1))/@4))))), @6=(@1*exp((@2*((sq((2.49935-w_0))/@3)+(sq((-1.8205-w_1))/@4))))), @7=(@1*exp((@2*((sq((8.24224-w_0))/@3)+(sq((-0.135562-w_1))/@4))))), @8=(@1*exp((@2*((sq((7.56746-w_0))/@3)+(sq((6.63626-w_1))/@4))))), @9=(@1*exp((@2*((sq((2.29742-w_0))/@3)+(sq((9.2219-w_1))/@4))))), @10=(@1*exp((@2*((sq((-6.28364-w_0))/@3)+(sq((9.36616-w_1))/@4))))), @11=(@1*exp((@2*((sq((-5.9367-w_0))/@3)+(sq((1.53678-w_1))/@4))))), @12=(@1*exp((@2*((sq((-4.80235-w_0))/@3)+(sq((-1.74116-w_1))/@4))))), @13=(@1*exp((@2*((sq((-4.53498-w_0))/@3)+(sq((-3.14509-w_1))/@4))))), @14=(@1*exp((@2*((sq((-5.7966-w_0))/@3)+(sq((-10.0445-w_1))/@4))))), @15=(@1*exp((@2*((sq((-0.376693-w_0))/@3)+(sq((-9.96826-w_1))/@4))))), @16=(@1*exp((@2*((sq((4.42149-w_0))/@3)+(sq

In [115]:
w.shape

(11, 1)

In [110]:
w = casadi.SX.sym('w', 11)

In [111]:
f1 = cs.Function('f',[x1,x2],[f(x1,x2)]);
f1(-9,0)

TypeError: f() missing 9 required positional arguments: 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', and 'x11'

In [112]:
x1 = cs.SX.sym('x_1', 1)
x2 = cs.SX.sym('x_2', 1)
x3 = cs.SX.sym('x_3', 1)
x4 = cs.SX.sym('x_4', 1)
x5 = cs.SX.sym('x_5', 1)
x6 = cs.SX.sym('x_6', 1)
x7 = cs.SX.sym('x_7', 1)
x8 = cs.SX.sym('x_8', 1)
x9 = cs.SX.sym('x_9', 1)
x10 = cs.SX.sym('x_10', 1)
x11 = cs.SX.sym('x_11', 1)

In [126]:
def f(x1, x2, x3, x4, x5, x6, x7, x8, x9 ,x10, x11):
    X = casadi.vertcat(x1, x2, x3, x4, x5, x6, x7, x8, x9 ,x10, x11)
    u0 = [0,0,0,0,0]
    mean = modelGP.continuous_model(X,u0)
    return mean
f1 = cs.Function('f',[x1, x2, x3, x4, x5, x6, x7, x8, x9 ,x10, x11],[f(x1, x2, x3, x4, x5, x6, x7, x8, x9 ,x10, x11)])
f1(-5,0,0,0,0,0,0,0,0,0,0)

this
[[x_1, x_2]]
(9, 1)
(2, 1)


DM([0, 0, 0, 2.79305, -0.000165787, 0, -0, -0, 0, 0, 0])

In [94]:
f1 = cs.Function('f',[x1,x2],[f(x1,x2)]);
f1(-9,0)

this
[[x_5, x_6]]
@1=0.30591, @2=-0.5, @3=30.9339, @4=4.78953, @5=(@1*exp((@2*((sq((0.623751-x_5))/@3)+(sq((1.67351-x_6))/@4))))), @6=(@1*exp((@2*((sq((2.49935-x_5))/@3)+(sq((-1.8205-x_6))/@4))))), @7=(@1*exp((@2*((sq((8.24224-x_5))/@3)+(sq((-0.135562-x_6))/@4))))), @8=(@1*exp((@2*((sq((7.56746-x_5))/@3)+(sq((6.63626-x_6))/@4))))), @9=(@1*exp((@2*((sq((2.29742-x_5))/@3)+(sq((9.2219-x_6))/@4))))), @10=(@1*exp((@2*((sq((-6.28364-x_5))/@3)+(sq((9.36616-x_6))/@4))))), @11=(@1*exp((@2*((sq((-5.9367-x_5))/@3)+(sq((1.53678-x_6))/@4))))), @12=(@1*exp((@2*((sq((-4.80235-x_5))/@3)+(sq((-1.74116-x_6))/@4))))), @13=(@1*exp((@2*((sq((-4.53498-x_5))/@3)+(sq((-3.14509-x_6))/@4))))), @14=(@1*exp((@2*((sq((-5.7966-x_5))/@3)+(sq((-10.0445-x_6))/@4))))), @15=(@1*exp((@2*((sq((-0.376693-x_5))/@3)+(sq((-9.96826-x_6))/@4))))), @16=(@1*exp((@2*((sq((4.42149-x_5))/@3)+(sq((-9.54355-x_6))/@4))))), @17=(@1*exp((@2*((sq((8.53755-x_5))/@3)+(sq((-9.72062-x_6))/@4))))), @18=(@1*exp((@2*((sq((3.65614-x_5))/@3)+(sq((

NotImplementedError: Wrong number or type of arguments for overloaded function '_vertcat'.
  Possible prototypes are:
    _vertcat([Sparsity])
    _vertcat([DM])
    _vertcat([SX])
    _vertcat([MX])
  You have: '(([int|SX],SX))'
